## Main Code

In [1]:
import numpy as np
from random import randint
from math import gcd, ceil
from qiskit import QuantumCircuit, transpile
from math import gcd
import numpy as np
from fractions import Fraction
from quantumrings.toolkit.qiskit import QrBackendV2

In [2]:
def generate_a(N, seed=1):
    """Generate a random integer 'a' such that gcd(a, N) = 1."""
    np.random.seed(seed)  # Set random seed for reproducibility
    
    a = randint(2, N)  # Random number between 2 and N-1
    while gcd(a, N) != 1:
        a = randint(2, N - 1)
    
    return a

In [3]:
def qft_dagger(num_qubits):
    """Creates the inverse Quantum Fourier Transform circuit."""
    qc = QuantumCircuit(num_qubits)
    for qubit in range(num_qubits // 2):
        qc.swap(qubit, num_qubits - qubit - 1)
    for j in range(num_qubits):
        for m in range(j):
            qc.cp(-np.pi / float(2 ** (j - m)), m, j)
        qc.h(j)
    return qc

In [4]:
def c_amodN(a, power, N):
    """Controlled multiplication by a mod N, where N is a semiprime."""
    if gcd(a, N) != 1:
        raise ValueError(f"a={a} must be coprime to N={N}.")
    
    n_qubits = len(bin(N)) - 2  # Minimum number of qubits to represent N
    U = QuantumCircuit(n_qubits)
    
    for _ in range(power):
        for qubit in range(n_qubits - 1):
            U.cx(qubit, (qubit + 1) % n_qubits)  # Placeholder for modular multiplication logic
    
    U = U.to_gate()
    U.name = f"{a}^{power} mod {N}"
    return U.control()

In [5]:
def qpe_amodN(a, N, provider, shots):
    """Performs quantum phase estimation on a*r mod N."""
    # Number of counting qubits
    N_COUNT = ceil(np.log2(N))
    n = ceil(np.log2(N))  # Number of qubits for auxiliary register
    
    # Create the quantum circuit
    qc = QuantumCircuit(N_COUNT + n, N_COUNT)
    
    # Initialize counting qubits to |+>
    for q in range(N_COUNT):
        qc.h(q)
    
    # Initialize the auxiliary register to |1>
    qc.x(N_COUNT)
    
    # Apply controlled-U operations
    for q in range(N_COUNT):
        qc.append(c_amodN(a, 2**q, N), [q] + [i + N_COUNT for i in range(n)])
    
    # Apply the inverse QFT
    qc.append(qft_dagger(N_COUNT), range(N_COUNT))
    
    # Measure the counting qubits
    qc.measure(range(N_COUNT), range(N_COUNT))
    
    # Transpile and execute
    backend = QrBackendV2(provider, num_qubits = qc.num_qubits)
    qc_transpiled = transpile(qc, backend)
    print(f"number of qubits: {qc_transpiled.num_qubits}")
    print(f"number of gate operations: {qc_transpiled.count_ops()}")
    job = backend.run(qc_transpiled, shots=shots)
    readings = job.result().get_memory()
    
    # Convert the measurement to phase
    most_common = max(set(readings), key=readings.count)
    phase = int(most_common, 2) / (2 ** N_COUNT)
    
    print(f"Register Reading: {most_common}")
    print(f"Corresponding Phase: {phase}")
    return phase


In [6]:
def find_factors(N, provider, shots=1024):
    """Shor's algorithm for finding non-trivial factors of N."""
    FACTOR_FOUND = False
    ATTEMPT = 0
    
    while not FACTOR_FOUND:
        ATTEMPT += 1
        print(f"\nATTEMPT {ATTEMPT}:")
        
        # Generate a random 'a' coprime to N
        a = generate_a(N)  
        print(f"Chosen 'a': {a}")
        
        # Perform QPE
        phase = qpe_amodN(a, N, provider, shots)  # Phase = s / r
        
        # Convert the phase to a rational approximation s/r
        frac = Fraction(phase).limit_denominator(N)
        r = frac.denominator
        print(f"Phase: {phase}, Approximated r = {r}")
        
        if r % 2 != 0 or pow(a, r // 2, N) in [0, 1, N - 1]:
            print("Invalid r, retrying...")
            continue  # Retry if r is odd or doesn't give non-trivial factors
        
        # Compute potential factors
        guesses = [gcd(pow(a, r // 2) - 1, N), gcd(pow(a, r // 2) + 1, N)]
        print(f"Guessed Factors: {guesses[0]} and {guesses[1]}")
        
        # Check the guesses
        for guess in guesses:
            if guess not in [1, N] and (N % guess) == 0:
                print(f"*** Non-trivial factor found: {guess} ***")
                FACTOR_FOUND = True
                return guess, N // guess  # Return the two factors
    
    print("No factors found.")
    return None, None

In [7]:
from QuantumRingsLib import QuantumRingsProvider

provider = QuantumRingsProvider(
    token='rings-200.tgGDmsBUuh8pqz21JO21ACa1bR6u6uZr',
    name='dignp5@yonsei.ac.kr'
)
shots = 256

provider.active_account()

{'name': 'dignp5@yonsei.ac.kr',
 'token': 'rings-200.tgGDmsBUuh8pqz21JO21ACa1bR6u6uZr',
 'max_qubits': '200'}

In [ ]:
from semiprimes import semiprimes

# Create the desired dictionary format
formatted_dict = {f"{period}": value for period, value in semiprimes.items()}

# Print the result
for period, semiprime in formatted_dict.items():
    print(f"{period}: {semiprime}")
    N = semiprime
    factor1, factor2 = find_factors(N, provider)
    print(f"\nFactors of {N}: {factor1}, {factor2}")
    print("--------------------------------------------------")

8: 143

ATTEMPT 1:
Chosen 'a': 107
number of qubits: 16
Register Reading: 11000000
Corresponding Phase: 0.75
Phase: 0.75, Approximated r = 4
Guessed Factors: 1 and 1

ATTEMPT 2:
Chosen 'a': 34
number of qubits: 16
Register Reading: 00100000
Corresponding Phase: 0.125
Phase: 0.125, Approximated r = 8
Invalid r, retrying...

ATTEMPT 3:
Chosen 'a': 10
number of qubits: 16
Register Reading: 10000000
Corresponding Phase: 0.5
Phase: 0.5, Approximated r = 2
Guessed Factors: 1 and 11
*** Non-trivial factor found: 11 ***

Factors of 143: 11, 13
--------------------------------------------------
10: 899

ATTEMPT 1:
Chosen 'a': 649
number of qubits: 20
Register Reading: 0110000000
Corresponding Phase: 0.375
Phase: 0.375, Approximated r = 8
Guessed Factors: 1 and 1

ATTEMPT 2:
Chosen 'a': 559
number of qubits: 20
Register Reading: 0010000000
Corresponding Phase: 0.125
Phase: 0.125, Approximated r = 8
Guessed Factors: 31 and 1
*** Non-trivial factor found: 31 ***

Factors of 899: 31, 29
-----------

## Extra codes for number of qubits, number of gate operations and the exection times

In [ ]:
%%time

N = 143
factor1, factor2 = find_factors(N, provider)
print(f"\nFactors of {N}: {factor1}, {factor2}")
print("--------------------------------------------------")


ATTEMPT 1:
Chosen 'a': 93
number of qubits: 16
number of gate operations: OrderedDict({'csx': 5355, 'cx': 3570, 'z': 3570, 'sdg': 1785, 'cp': 28, 'h': 16, 'measure': 8, 'x': 1})
Register Reading: 01100000
Corresponding Phase: 0.375
Phase: 0.375, Approximated r = 8
Guessed Factors: 1 and 1

ATTEMPT 2:
Chosen 'a': 84
number of qubits: 16
number of gate operations: OrderedDict({'csx': 5355, 'cx': 3570, 'z': 3570, 'sdg': 1785, 'cp': 28, 'h': 16, 'measure': 8, 'x': 1})
Register Reading: 10000000
Corresponding Phase: 0.5
Phase: 0.5, Approximated r = 2
Guessed Factors: 1 and 1

ATTEMPT 3:
Chosen 'a': 90
number of qubits: 16
number of gate operations: OrderedDict({'csx': 5355, 'cx': 3570, 'z': 3570, 'sdg': 1785, 'cp': 28, 'h': 16, 'measure': 8, 'x': 1})
Register Reading: 10000000
Corresponding Phase: 0.5
Phase: 0.5, Approximated r = 2
Guessed Factors: 1 and 13
*** Non-trivial factor found: 13 ***

Factors of 143: 13, 11
--------------------------------------------------
CPU times: user 41.9 s

In [ ]:
%%time

N = 899
factor1, factor2 = find_factors(N, provider)
print(f"\nFactors of {N}: {factor1}, {factor2}")
print("--------------------------------------------------")


ATTEMPT 1:
Chosen 'a': 402
number of qubits: 20
number of gate operations: OrderedDict({'csx': 27621, 'cx': 18414, 'z': 18414, 'sdg': 9207, 'cp': 45, 'h': 20, 'measure': 10, 'x': 1})
Register Reading: 0111000000
Corresponding Phase: 0.4375
Phase: 0.4375, Approximated r = 16
Guessed Factors: 31 and 1
*** Non-trivial factor found: 31 ***

Factors of 899: 31, 29
--------------------------------------------------
CPU times: user 2min 40s, sys: 112 ms, total: 2min 40s
Wall time: 1min 9s


In [ ]:
%%time

N = 3127
factor1, factor2 = find_factors(N, provider)
print(f"\nFactors of {N}: {factor1}, {factor2}")
print("--------------------------------------------------")


ATTEMPT 1:
Chosen 'a': 1212
number of qubits: 24
number of gate operations: OrderedDict({'csx': 135135, 'cx': 90090, 'z': 90090, 'sdg': 45045, 'cp': 66, 'h': 24, 'measure': 12, 'x': 1})
Register Reading: 011100000000
Corresponding Phase: 0.4375
Phase: 0.4375, Approximated r = 16
Guessed Factors: 1 and 1

ATTEMPT 2:
Chosen 'a': 705
number of qubits: 24
number of gate operations: OrderedDict({'csx': 135135, 'cx': 90090, 'z': 90090, 'sdg': 45045, 'cp': 66, 'h': 24, 'measure': 12, 'x': 1})
Register Reading: 101100000000
Corresponding Phase: 0.6875
Phase: 0.6875, Approximated r = 16
Guessed Factors: 1 and 1

ATTEMPT 3:
Chosen 'a': 2999
number of qubits: 24
number of gate operations: OrderedDict({'csx': 135135, 'cx': 90090, 'z': 90090, 'sdg': 45045, 'cp': 66, 'h': 24, 'measure': 12, 'x': 1})
Register Reading: 110000000000
Corresponding Phase: 0.75
Phase: 0.75, Approximated r = 4
Guessed Factors: 1 and 1

ATTEMPT 4:
Chosen 'a': 146
number of qubits: 24
number of gate operations: OrderedDict(

In [ ]:
%%time

N = 11009
factor1, factor2 = find_factors(N, provider)
print(f"\nFactors of {N}: {factor1}, {factor2}")
print("--------------------------------------------------")


ATTEMPT 1:
Chosen 'a': 1758
number of qubits: 28
number of gate operations: OrderedDict({'csx': 638937, 'cx': 425958, 'z': 425958, 'sdg': 212979, 'cp': 91, 'h': 28, 'measure': 14, 'x': 1})


In [ ]:
%%time

N = 47053
factor1, factor2 = find_factors(N, provider)
print(f"\nFactors of {N}: {factor1}, {factor2}")
print("--------------------------------------------------")


ATTEMPT 1:
Chosen 'a': 3402
number of qubits: 32
number of gate operations: OrderedDict({'csx': 2949075, 'cx': 1966050, 'z': 1966050, 'sdg': 983025, 'cp': 120, 'h': 32, 'measure': 16, 'x': 1})


In [ ]:
%%time

N = 167659
factor1, factor2 = find_factors(N, provider)
print(f"\nFactors of {N}: {factor1}, {factor2}")
print("--------------------------------------------------")


ATTEMPT 1:
Chosen 'a': 120048
